In [2]:
import numpy as np
def accuracy (y_hat, y):
    assert(y_hat.size == y.size)
    y_hat = np.array(y_hat)
    y = np.array(y)
    X = np.where(y==y_hat,1,0)
    return np.sum(X)/len(X)
    pass

In [15]:
#1a,b
from autograd.numpy import exp,log
import pandas as pd
import matplotlib.pyplot as plt
import autograd.numpy as np 
from autograd import grad 
import math
import random
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from matplotlib.animation import FuncAnimation

# def gradient(theta,X,y):
#         X = np.array(X)
#         y = np.array(y)
#         theta = np.array(theta)
#         print("grad",theta)
#         return (y*(log((1.0/(1.0 + exp(-(X.dot(theta))))))) +  (1 - y)*(log(1 - (1.0/(1.0 + exp(-(X.dot(theta))))))))

class LogisticRegression():
    def __init__(self, fit_intercept=True):
        
        self.samantha = 0
        self.fit_intercept = fit_intercept
        self.coef_ = None #Replace with numpy array or pandas series of coefficients learned using using the fit methods

        pass

    def loss(self,w, X, y):
        n = float(len(X))
        y_pred = np.dot(X, w)
        # return -((np.sum(-(y_pred * y) + y*log(1.0 + exp(y_pred))) / n ))
        # return -((np.sum(-(y_pred * y) + log(1.0 + exp(y_pred))) / n) + self.samantha*(np.sum(np.abs(w))))
        return -((np.sum(-(y_pred * y) + log(1.0 + exp(y_pred))) / n ) + self.samantha*(np.sum(np.dot(w.T,w))))
    def sigmoid(self,x):
        return (1/(1 + exp(-x)))

    def __init__(self, fit_intercept=True):
        
        self.samantha = 0
        self.fit_intercept = fit_intercept
        self.coef_ = None #Replace with numpy array or pandas series of coefficients learned using using the fit methods

        pass

    def fit_regularised(self, X, y,batch_size = None, n_iter=200, lr=0.01, lr_type='constant'):
       
        if (self.fit_intercept):
            bias = pd.DataFrame(pd.Series([1.0 for i in range(len(X))]))
            X = pd.concat([bias,X],axis=1)
        col_len = len(X.columns)
        coef_ = np.zeros(col_len)
        for i in range(1,n_iter):
            cur =  coef_.copy()
            coef_ = cur - lr*((X.T).dot((self.sigmoid(X.dot(cur))) - y))       
        self.coef_ = coef_
        return coef_
        pass

    def fit_regularised_autograd(self, X, y, batch_size=None, n_iter=200, lr=0.01, lr_type='constant'):

        if (self.fit_intercept):
            bias = pd.DataFrame(pd.Series([1.0 for i in range(len(X))]))
            X = pd.concat([bias,X],axis=1)
        col_len = len(X.columns)
        X = np.array(X)
        y = np.array(y)
        coef_ = np.zeros(col_len)
        gradient = grad(self.loss)
        # change according to regularization
        for i in range(1,n_iter):
            cur =  coef_.copy()
            coef_ = cur + lr*(gradient(cur,X,y))       
        self.coef_ = coef_
        return coef_
        pass
        
    def predict(self, X):
     
        X_ = X.copy()
        if (self.fit_intercept):
            bias = pd.DataFrame(pd.Series([1.0 for i in range(len(X_))]))
            X_ = pd.concat([bias,X_],axis=1) 
        k = self.sigmoid(np.dot(X_,self.coef_))
        p = np.where(k<0.5,0,1)
        return pd.Series(p)
        pass
    def softmax(self,X):
        t = []
        for i in range(len(self.coef_)):
            t.append(exp(np.dot(X,self.coef_[i])))
        t = np.array(t)
        return t/np.sum(t)

    def k_class_predict(self, X,flag = False): 
        X = np.array(X)
        soft=np.vectorize(self.softmax,signature='(n)->(m)')
        r = soft(X)
        if flag == True:
            return r
        else:
            return np.argmax(r,axis = 1)
        pass


    def confusion(self,X,y):
        k = len(self.coef_)
        m = np.zeros((k,k))
        y_hat = self.k_class_predict(X,False)
        for i in range(len(y_hat)):
            m[y_hat[i],y[i]] += 1
        return m
    def fit_k_class_regularised(self, X, y,batch_size = None, n_iter=200, lr=0.01, lr_type='constant'):
       
        if batch_size==None:
            batch_size=len(X)
        self.batch_size=batch_size
        self.n_iter = n_iter
        k = len(np.unique(y))
        n = len(X)
        batch_size = len(X)
        temp = lr 
        column_length = len(X.columns)
        theta = np.zeros((k,column_length))
        self.coef_ = theta
        soft = self.k_class_predict(X)
        for i in range(1,n_iter):
            current =  theta.copy()
            for j in range(k):
                theta[j] = current[j] + lr*(np.sum(X*(np.tile(np.where(y==j,1,0) - soft[:,j],(len(current[0]),1)).T),axis=0))
                   
        self.coef_ = theta
        pass
    def plot(self,X,y):
    
        b = self.coef_[0]
        w1, w2 = self.coef_[1:]
        c = -b/w2
        m = -w1/w2
        xmin, xmax = -3, 3
        ymin, ymax = -3, 3
        xd = np.array([xmin, xmax])
        yd = m*xd + c
        plt.figure()
        plt.plot(xd, yd, 'k', lw=1, ls='--')
        plt.fill_between(xd, yd, ymin, color='tab:blue', alpha=0.2)
        plt.fill_between(xd, yd, ymax, color='tab:orange', alpha=0.2)
        plt.scatter(X[y==0][:,0],X[y==0][:,1])
        plt.scatter(X[y==1][:,0],X[y==1][:,1])
        plt.xlim(xmin, xmax)
        plt.ylim(ymin, ymax)
        plt.ylabel(r'$x_2$')
        plt.xlabel(r'$x_1$')
        plt.show()
        
        pass


In [11]:
def optimum_lamda(X,y):
    LR = LogisticRegression(fit_intercept=True)
    alpha = 0.01
    maxlamda = 0
    maxaccu = 0
    for i in range (0,100):
        LR.samantha = alpha*i
        LR.fit_regularised_autograd(pd.DataFrame(X),pd.Series(y))
        y_hat = LR.predict(pd.DataFrame(X))
        accu = accuracy(y_hat,y)
        if accu >= maxaccu :
            maxaccu = accu
            maxlamda = alpha*i
    return maxlamda           





In [17]:
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold

x = load_breast_cancer(as_frame=True)
X = x.data
minmax = MinMaxScaler()
X[list(X)]= minmax.fit_transform(X)
y = x.target
X = np.array(X)
y = np.array(y)
#for 2b
print(optimum_lamda(X,y))
k = KFold(n_splits=3)
accuracies = []

for train,test in k.split(X,y):
    batch_size = 5
    fit_intercept = True
    LR = LogisticRegression(fit_intercept=fit_intercept)
    LR.fit_regularised(pd.DataFrame(X[train]), pd.Series(y[train]),batch_size)
    y_hat = LR.predict(pd.DataFrame(X[test]))
    accuracies.append(accuracy(y_hat,y[test]))
print(accuracies)

np.random.seed(42)
N = 30
P = 2
X = pd.DataFrame(np.random.randn(N, P))
y = pd.Series([0,0,0,0,0,0,1,1,1,0,0,0,0,1,1,0,0,0,0,0,0,1,1,1,0,0,0,0,1,1])

LR = LogisticRegression(fit_intercept = False)
LR.fit_k_class_regularised(pd.DataFrame(X),pd.Series(y))
y_hat = LR.k_class_predict(pd.DataFrame(X))
print(y_hat,y)

# LR.plot(np.array(X),np.array(y))


C:\Users\shahi\AppData\Local\Programs\Python\Python37\Lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\shahi\AppData\Local\Programs\Python\Python37\Lib\site-packages\pandas\core\frame.py:3076: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.iloc._setitem_with_indexer((slice(None), indexer), value)
C:\Users\shahi\AppData\Local\Programs\Python\Python37\Lib\site-packages\pandas\core\frame.py:3041: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the doc

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [1]:

# NETS package
import nets
from nets.nn.modules import Module
from nets.nn.activation import *


class DNN(Module):
    """
    Dense Neural Network.
    Attributes:
        layer_dimensions (list(int)): ``list`` of ``int`` containing all layer dimensions. The dimension at index ``i``
            is the value of ``layer_dimensions`` at this index.
        hidden_dimensions (list(int)): ``list`` of ``int`` containing only hidden dimensions.
        training (bool): Boolean to indicate if we are training or not. This function can namely be
            used for inference only, in which case we do not need to store the features
            values.
        activation_hidden (Activation): activation function used in hidden layers.
        activation_output (Activation): activation function used in the last layer.
        _parameters (dict): ``dict`` containing values of weights and biases for each layers.
            - ``weight_{i}``: contains the weight matrix of layer ``i``
            - ``bias_{i}``: contains the bias array of layer ``i``
        _cache (dict): ``dict`` with
            - the linear combinations Z^[l] = W^[l]a^[l-1] + b^[l] for l in [1, L].
            - the activations A^[l] = activation(Z^[l]) for l in [1, L].
            We cache them in order to use them when computing gradients in the back propagation.
    """

    def __init__(self, layer_dimensions, activation_hidden=ReLU()):
        super().__init__()
        # Add layers
        self.layer_dimensions = layer_dimensions
        self.hidden_dimensions = layer_dimensions[1: -1]
        # Add activation functions
        assert isinstance(activation_hidden,
                          Activation), "unrecognized activation function type"
        self.activation_hidden = activation_hidden
        self.activation_output = Softmax(axis=1)
        # Add weights and biases
        self.init_parameters()

    def init_parameters(self):
        """Initialize the parameters dictionary.
        For Dense Neural Network, the parameters are either weights or biases. They are saved in the dictionary
        with the following keys:
            - w_{i}: weight matrix at layer i,
            - b_{i}: bias vector at layer i.
        """
        # Create the weights and biases
        for i in range(1, len(self.layer_dimensions)):
            # Initialization from He et al.
            mu = 0
            var = 2 / self.layer_dimensions[i]
            sigma = np.sqrt(var)
            weight_shape = (
                self.layer_dimensions[i - 1], self.layer_dimensions[i])
            weight = np.random.normal(loc=mu, scale=sigma, size=weight_shape)
            bias = np.zeros((self.layer_dimensions[i], ))

            # Saving in the parameters dict
            layer_weight = "w_" + str(i)
            self._parameters[layer_weight] = weight
            layer_b = "b_" + str(i)
            self._parameters[layer_b] = bias

    def forward(self, inputs):
        """One forward step.
        Args:
            inputs (numpy.array): float numpy array with shape (n^[0], batch_size). Input image batch.
        Returns:
            outputs (numpy.array): float numpy array with shape (n^[L], batch_size). The output predictions of the
                network, where n^[L] is the number of prediction classes. For each input i in the batch,
                output[c, i] gives the probability that input ``i`` belongs to class ``c``.
        """
        depth = len(self.layer_dimensions) - 1
        z = inputs
        # Add the outputs to the features
        if self.training:
            layer_a = "a_0"
            self._cache[layer_a] = z
        # 1/ Iterates through the depth of the neural network
        for i in range(1, depth + 1):
            # 1.1/ Get the weights and biases from the params
            layer_w = "w_" + str(i)
            layer_b = "b_" + str(i)
            weight = self._parameters[layer_w]
            bias = self._parameters[layer_b]
            # 1.2/ Compute the outputs
            z = np.dot(z, weight) + bias
            # Add the outputs to the features
            if self.training:
                layer_z = "z_" + str(i)
                self._cache[layer_z] = z
            # 2/ Hidden Activation
            # The activation only occurs in the hidden layers
            if i < depth:
                z = self.activation_hidden(z)
                # Add the outputs to the features
                if self.training:
                    layer_a = "a_" + str(i)
                    self._cache[layer_a] = z
        # 3/ Output
        outputs = self.activation_output(z)
        # Add the outputs to the features
        if self.training:
            layer_a = "a_" + str(depth)
            self._cache[layer_a] = outputs

        return outputs
    def fit(self, X, y, batch_size=None, n_iter = 200, lr=0.01, lr_type='constant'):
        self.init_parameters()

        # if (self.fit_intercept):
        #     bias = pd.DataFrame(pd.Series([1.0 for i in range(len(X))]))
        #     X = pd.concat([bias,X],axis=1)
        # col_len = len(X.columns)
        X = np.array(X)
        y = np.array(y)
        # coef_ = np.zeros(col_len)
        # gradient = grad(self.loss)
        # change according to regularization
        for i in range(1,n_iter):
            for j in range (len(X)):
                
                y_ = self.forward(X)
                self.backward(y_, y)
            # cur =  coef_.copy()
            # coef_ = cur + lr*(gradient(cur,X,y))
                   
        # self.coef_ = coef_
        # return coef_
        pass
    def nnpredict(self, X):
        arr = []
        for i in range (len(X)):



            arr = arr.append(self.forward(i))
        return arr


    def backward(self, outputs, labels):


        # Layers & shape
        depth = len(self.layer_dimensions) - 1
        batch_size, num_classes = outputs.shape
        coefficient = 1 / batch_size
        # 1/ First case: last layer -> output
        layer_a = "a_" + str(depth - 1)
        a = self._cache[layer_a]
        Jz = outputs - labels
        # Weights gradients
        dw = coefficient * np.dot(a.T, Jz)
        db = coefficient * np.sum(Jz, axis=0)
        self._grad["dw_" + str(depth)] = dw
        self._grad["db_" + str(depth)] = db
        # 2/ Second case: inside the layers
        for i in range(depth - 1, 0, -1):
            # Get the weights and biases
            layer_w = "w_" + str(i + 1)
            layer_a = "a_" + str(i - 1)
            layer_z = "z_" + str(i)
            w = self._parameters[layer_w]
            a = self._cache[layer_a]
            z = self._cache[layer_z]
            # Gradients
            Jz = self.activation_hidden.backward(z) * np.dot(Jz, w.T)
            db = coefficient * np.sum(Jz, axis=0)
            dw = coefficient * np.dot(a.T, Jz)
            self._grad["dw_" + str(i)] = dw
            self._grad["db_" + str(i)] = db


ModuleNotFoundError: No module named 'nets'

In [20]:
np.random.seed(42)
N = 30
P = 2
X = pd.DataFrame(np.random.randn(N, P))
y = pd.Series([0,0,0,0,0,0,1,1,1,0,0,0,0,1,1,0,0,0,0,0,0,1,1,1,0,0,0,0,1,1])

LR = LogisticRegression(fit_intercept = False)
LR.fit(pd.DataFrame(X),pd.Series(y))
y_hat = LR.nnpredict(pd.DataFrame(X))
print(y_hat,y)


AttributeError: 'LogisticRegression' object has no attribute 'fit'

In [2]:
import nets 

ModuleNotFoundError: No module named 'nets'

In [ ]:
# baseline model for the dogs vs cats dataset
import sys
from matplotlib import pyplot
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
 
# define cnn model
def define_model():
	model = Sequential()
    #model = vgg1()
    #model = vgg2 ()
    #model = transfer()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 3)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
	return model
 
# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	# filename = sys.argv[0].split('/')[-1]
	pyplot.savefig('vgg1' + '_plot.png')
	pyplot.close()
 
# run the test harness for evaluating a model
def run_test_harness():
	# define model
	model = define_model()
	# create data generator
	datagen = ImageDataGenerator(rescale=1.0/255.0)
	# prepare iterators
	train_it = datagen.flow_from_directory('images/train/',
		class_mode='binary', batch_size=64, target_size=(200, 200))
	test_it = datagen.flow_from_directory('images/test/',
		class_mode='binary', batch_size=64, target_size=(200, 200))
	# fit model
	history = model.fit_generator(train_it, steps_per_epoch=len(train_it),
		validation_data=test_it, validation_steps=len(test_it), epochs=20, verbose=0)
	# evaluate model
	_, acc = model.evaluate_generator(test_it, steps=len(test_it), verbose=0)
	print('> %.3f' % (acc * 100.0))
	# learning curves
	summarize_diagnostics(history)
 
# entry point, run the test harness
run_test_harness()